# Hands-On-Lab: Datenanalyse mit dem DNBLab

## Inhalt dieses Workshops: 

  1.  Willkommen
  2.  Sammlung konkreter Anliegen der Teilnehmer*innen
  3.  Einführung in das Thema
  4.  Einrichtung Workspace 
  5.  Gemeinsame exemplarische Aufbereitung eines Datensets mit beispielhafter Analyse
  6.  Erstellung eines eigenen Datensets mit dem DNB SRU Query Tool
  7.  Gruppen- oder Einzelarbeit an weiteren Fragestellungen
  8.  Sammlung der Ergebnisse und Erkenntnisse
  9.  Abschlussdiskussion
  
  Einführung in das Thema Datenanalyse und möglicher Mehrwerte
 

### 3. Sammlung konkreter Anliegen

https://tinyurl.com/dnblab-ws1225-sammlung

### 4. Gemeinsame Aufbereitung und erste Analyse eines Datensets

Bitte folgendes Datenset nutzen `/shared/dataset_turorial_eco.xml` oder hier herunterladen: https://github.com/deutsche-nationalbibliothek/dnblab/blob/main/dataset_tutorial_eco.xml

Wichtigste MARC21-xml-Felder: https://www.dnb.de/SharedDocs/Downloads/DE/Professionell/Services/efa2023HandoutInhalteInMarc.pdf?__blob=publicationFile&v=2


#### Bibliotheken importieren und Arbeitsumgebung einrichten

Zuerst werden die notwendigen Bibliotheken importiert:

  - `pandas`: Biblikothek für die Datenmanipulation
  - `lxml` bzw. etree: für das Parsen von XML-Daten
  - `unicodedata`: Für die Arbeit mit Unicode-Zeichen und -Strings
  - `plotly.express`: leistungsstarke Bibliothek in Python für die Erstellung von interaktiven Visualisierungen und Diagrammen


In [ ]:
# Benötigte Python-Bibliotheken importieren
import pandas as pd
from lxml import etree
import plotly.express as px

In [1]:
with open ("dataset_tutorial_eco.xml", encoding="utf-8") as f: 
    xml = f.read()

In [ ]:
print(xml[:1000])

In [ ]:
#df = pd.read_xml("shared/dataset_tutorial_eco.xml", xpath="//marc:controlfield", namespaces={"marc":"http://www.loc.gov/MARC21/slim"})
#df

In [ ]:
# Laden der MARC-xml-Datei in ElementTree: 
tree = etree.parse('shared/dataset_tutorial_eco.xml')

# Laden des Root-Verzeichnisses des XML:
root = tree.getroot()                

# Definiere den Namensraum für MARC21 - muss bei Bedarf angepasst werden (wenn bspw. ein anderes Metadatenformat genutzt wird):
ns = {'marc': 'http://www.loc.gov/MARC21/slim'} 
# Einlesen der einzelnen enthaltenen Datensätze (records) in eine Liste: 
records = root.findall('.//marc:record', namespaces=ns)

#Ausgabe der Länge der Liste als Kontrolle für die Anzahl der enthaltenen Datensätze:
print("Gefundene Records:", len(records))

#### Direkter Zugriff auf die Inhalte des XML

Im Folgenden wird beispielhaft einer der Datensätze aus der Variable "records" separat gespeichert. Daraufhin wird er in einen String umgewandelt, um ihn betrachten zu können. Im Anschluss wird aus diesem Datensatz das "Controlfield" mit dem Tag 001 extrahiert und dessen Inhalt exemplarisch ausgegeben.


In [ ]:
# Beispielhaft einen der Datensätze aus der Liste als eigene Variable speichern:
test_record = records[0]

# In einen String umwandeln: 
test_record_string = etree.tostring(test_record, encoding="utf-8")
print(test_record_string)

Extrahieren des Inhalts des Controlfields mit dem Tag "001" aus "test_record":

In [ ]:
# Angabe des Namespaces: 
ns = {'marc': 'http://www.loc.gov/MARC21/slim'}

# Extraktion des Elements "Controlfield" mit dem Tag "001"
controlfield_001 = test_record.find("marc:controlfield[@tag='001']", namespaces=ns)
print(controlfield_001)

In [ ]:
# Ausgabe des Inhalts des Controlfields: 
print(controlfield_001.text)

#### Funktion zur Extraktion mehrerer Inhalte aus den einzelnen Datensätzen:

Die Funktion `parse_record` erwartet nun immer einen einzelnen Datensatz (record), den sie verarbeitet. Hierzu ist zunächst wieder der passende Namespace defininiert.

Zusätzlich wurde eine weitere Funktion `extract_text` definiert. Diese erwartet eine Pfadangabe zu einem XML-Element wie bspw. `marc:controlfield[@tag='001']` oder auch `marc:datafield[@tag='245']/marc:subfield[@code='a']`. Die Funktion sucht dann nach den entsprechenden Elementen im XML und extrahiert den Inhalt. Falls mehrere Felder mit demselben Pfad enthalten sind, werden die Inhalte durch "; " zu einem String verbunden.

Anschließend wird für jeden gewünschten Inhalt eine Variable definiert (bspw. "titel"), die dann auf die Funktion `extract_text` zugreift, dieser den angegebenen Pfad übergibt und den zurückgegebenen Inhalt speichert.

Zuletzt werden die Inhalte jeder erstellten Variable einem passenden Schlüssel zugeordnet und als einfaches Dictionary (Sammlung von Schlüssel-Werte-Paaren) zurückgegeben.

In [ ]:
# Funktion zum Extrahieren von Datensätzen
def parse_record(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    
    def extract_text(xpath_query):
        fields = record.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")
    titel = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    author = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    author_rela = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='e']")
    add_author = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='a']")
    add_author_rela = extract_text("marc:datafield[@tag='700']/marc:subfield[@code='e']")
    jahr = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='c']")

    return {
        "idn": idn,
        "author": author,
        "titel": titel,
        "author_rela": author_rela,
        "additional_author": add_author,
        "added_rela": add_author_rela,
        "jahr": jahr        
    }

Nun werden die einzelnen Datensätze aus der Menge der gesammelten Datensätze in der Variable `records` nacheinander (Datensatz für Datensatz) der Funktion `parse_record` übergeben und die zurückgewonnenen Inhalte in der Variable `result` gespeichert:

In [ ]:
# Übergabe der einzelnen Datensätze an die Funktion "parse_record":
result = [parse_record(record) for record in records]

In [ ]:
print(result)

Abschließend können die Inhalte aus der Variable result in eine Tabelle in Form eines Pandas-Dataframes für die weitere Arbeit umgewandelt werden:

In [ ]:
# Umwandlung in ein Pandas-Dataframe
df = pd.DataFrame(result)
df#.head()

#### Speichern des Dataframes

Dataframes können in mehreren Formaten gespeichert werden:

Speichern als CSV, Excel oder hdf:

In [ ]:
df.to_csv("Publiaktionsdaten.csv", encoding="utf-8")

In [ ]:
df.to_excel("Publiaktionsdaten.xlsx")

In [ ]:
df.to_hdf("Publiaktionsdaten.h5", key="df")

#### Erste Analyse

In [ ]:
# Häufigkeit der Autoren zählen
author_counts = df['author'].value_counts().head(15).reset_index()

In [ ]:
#Augabe im Dataframe
author_counts.columns=['author', 'count']
author_counts

In [ ]:
# Balkendiagramm erstellen
fig1 = px.bar(author_counts, x='author', y='count', height=550,
             title='Häufigste Autoren im Datenset', color='count', color_continuous_scale='Viridis')

# Diagramm anzeigen
fig1.show()

### 5. Selbst aktiv werden

https://github.com/deutsche-nationalbibliothek/sruquerytool 

Alternative (nur bis 20.000 Datensätze geeignet):  
https://portal.dnb.de und 
https://srudump.streamlit.app/


### 6. Sammlung Ergebnisse

https://tinyurl.com/dnblab-ws1225-sammlung